# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [3]:
from trading_bot.agent import Agent

model_name = 'model_GOOG_50'
test_stock = 'data/GOOG_2019.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [4]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


### 3. Running Eval

In [8]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG switching to TensorFlow for CPU
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1057.19
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1080.97
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Sell at: $1089.90 | Position: +$32.71
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Sell at: $1098.26 | Position: +$17.29
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1073.90
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Sell at: $1090.99 | Position: +$17.09
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1098.71
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1095.06
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1095.01
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Sell at: $1121.37 | Position: +$22.66
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1120.16
2020-01-11 01:09:29 192.168.1.14 root[4294] DEBUG Buy at: $1113.65
2020-01-11 01:09:29 192.168.1.

### 4. Visualize

In [9]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [10]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)